In [132]:
!pip3 install yfinance

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 62 kB 1.3 MB/s 
  Attempting uninstall: requests
    Found existing installation: requests 2.23.0
    Uninstalling requests-2.23.0:
      Successfully uninstalled requests-2.23.0


In [133]:
import joblib
import datetime
import numpy as np
import pandas as pd
import seaborn as sns
import yfinance as yf
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler

In [137]:
def processData(coin):
  #Reading data
  BASE_URL = '/content/drive/MyDrive/data/'
  data = pd.read_csv(BASE_URL + coin + '.csv')

  #Creating date
  data['Date'] = pd.to_datetime(data['timestamp'], unit = 's').dt.date

  #Intializing variable
  dates = data['Date'].unique()
  dates = list(dates)
  for i in range(len(dates)):
    dates[i] = str(dates[i])

  #Extracting ['Date', 'Close'] and setting 'date' as index
  data = data[['Close', 'Date']]
  data['Date'] = data['Date'].astype(str)

  final_data = []

  #Creating day-wise dataframe for the given data
  for i in dates:
    max = data[data['Date'] == i][-1:]
    final_data.append(max)

  #Reshape data to 2D
  final_data = np.array(final_data).reshape(len(final_data), 2)

  #Converting to DataFrame
  cols = data.columns
  final_data = pd.DataFrame(final_data, columns = cols)

  return final_data.set_index('Date', drop = True)

def preprocessData(dataCleaned):
  data_matrix = []
  col = 10

  # Normalizing data
  scaler = MinMaxScaler()
  dataCleaned['Close'] = dataCleaned['Close'].astype(float)
  dataCleaned['Close'] = scaler.fit_transform(np.array(dataCleaned['Close']).reshape(-1,1))

  for i in range(len(dataCleaned) - col+1):
    data_matrix.append(dataCleaned[i:i+col].values)

  row = int(len(data_matrix) * 0.7)
  dataCleaned = np.asarray(data_matrix)

  # # Splitting data
  X_train, y_train = dataCleaned[:row, :-1], dataCleaned[:row, -1]
  X_test, y_test = dataCleaned[row:, :-1], dataCleaned[row:, -1]

  # Reshaping feature variables
  # 1. X_train
  shape = list(X_train.shape)
  totShape = np.prod(shape)
  main = int(totShape/9)

  X_train = np.array(X_train).reshape(main, 1, 9)

  # 2. X_test
  shape = list(X_test.shape)
  totShape = np.prod(shape)
  main = int(totShape/9)

  X_test = np.array(X_test).reshape(main, 1, 9)

  return scaler, X_train, y_train, X_test, y_test

def model_train(X_train, y_train, coin):
  model = tf.keras.models.Sequential()
  model.add(tf.keras.layers.LSTM(units = 30, input_shape = (None, 9), return_sequences = True))
  model.add(tf.keras.layers.Dense(units = 32, activation = 'linear'))
  model.add(tf.keras.layers.LSTM(units = 30, return_sequences = False))
  model.add(tf.keras.layers.Dense(units = 1, activation = 'linear'))

  model.compile(optimizer = 'adam', loss = 'mean_squared_error', metrics = ['mse'])

  model.fit(X_train, y_train, batch_size = 5, epochs = 25, validation_split = .02)
  
  path = f'/content/drive/MyDrive/model/{coin}.h5'
  joblib.dump(model, path)
  
  return f'Model saved successfully at {path}!'

def predict_NextDay(scaler, coin):
  MODEL_PATH = f'/content/drive/MyDrive/model/{coin}.h5' 
  model = joblib.load(MODEL_PATH)
  coins_dict = {
    'dogecoin' : 'DOGE-USD',
    'bitcoin' : 'BTC-USD',
    'ethereum' : 'ETH-USD',
    'iota' : 'MIOTA-USD',
    'tron' : 'TRX-USD'
  }

  ticker = coins_dict[coin]

  today = datetime.date.today()
  d1 = today.strftime("%Y-%m-%d")
  d2 = datetime.date.today() - datetime.timedelta(days = 9)
  d2 = d2.strftime("%Y-%m-%d")

  new_sample = yf.download(ticker, start = d2, end = d1, progress = False)

  last9 = new_sample[['Close']]
  vals = last9.values

  temp = []

  for i in range(len(vals)):
    temp.append(vals[0][0])

  sample = scaler.fit_transform(np.array(temp).reshape(-1, 1))
  sample = np.asarray(sample)

  sample = np.reshape(sample, (1, 1, 9))

  prediction = model.predict(sample, batch_size = 2)

  prediction = scaler.inverse_transform(prediction)
  return prediction

In [135]:
coins_dict = {
    'dogecoin' : 'DOGE-USD',
    'bitcoin' : 'BTC-USD',
    'ethereum' : 'ETH-USD',
    'iota' : 'MIOTA-USD',
    'tron' : 'TRX-USD'
}

In [136]:
coins_dict['dogecoin']

'DOGE-USD'

In [67]:
data = processData('dogecoin')

In [124]:
scaler, X_train, y_train, X_test, y_test = preprocessData(data)

In [125]:
model_train(X_train, y_train, 'dogecoin')

Epoch 1/25
122/122 [==============================] - 5s 12ms/step - loss: 7.6358e-07 - mse: 7.6358e-07 - val_loss: 8.0693e-06 - val_mse: 8.0693e-06
Epoch 2/25
122/122 [==============================] - 1s 4ms/step - loss: 7.0687e-07 - mse: 7.0687e-07 - val_loss: 4.3973e-06 - val_mse: 4.3973e-06
Epoch 3/25
122/122 [==============================] - 1s 5ms/step - loss: 6.2077e-07 - mse: 6.2077e-07 - val_loss: 1.1066e-05 - val_mse: 1.1066e-05
Epoch 4/25
122/122 [==============================] - 1s 4ms/step - loss: 6.4684e-07 - mse: 6.4684e-07 - val_loss: 6.5045e-06 - val_mse: 6.5045e-06
Epoch 5/25
122/122 [==============================] - 0s 4ms/step - loss: 6.6727e-07 - mse: 6.6727e-07 - val_loss: 5.1338e-06 - val_mse: 5.1338e-06
Epoch 6/25
122/122 [==============================] - 1s 4ms/step - loss: 5.8311e-07 - mse: 5.8311e-07 - val_loss: 8.2717e-06 - val_mse: 8.2717e-06
Epoch 7/25
122/122 [==============================] - 1s 4ms/step - loss: 5.4847e-07 - mse: 5.4847e-07 - val_lo

'Model saved successfully at /content/drive/MyDrive/model/dogecoin.h5!'

In [138]:
predict_NextDay(scaler, 'dogecoin')

array([[0.08343949]], dtype=float32)

In [139]:
#Bitcoin
coin = 'bitcoin'
data = processData(coin)
scaler, X_train, y_train, X_test, y_test = preprocessData(data)
model_train(X_train, y_train, coin)
predict_NextDay(scaler, coin)

Epoch 1/25
186/186 [==============================] - 7s 9ms/step - loss: 6.1435e-04 - mse: 6.1435e-04 - val_loss: 2.3385e-04 - val_mse: 2.3385e-04
Epoch 2/25
186/186 [==============================] - 1s 5ms/step - loss: 1.0391e-04 - mse: 1.0391e-04 - val_loss: 1.5189e-04 - val_mse: 1.5189e-04
Epoch 3/25
186/186 [==============================] - 1s 5ms/step - loss: 8.5002e-05 - mse: 8.5002e-05 - val_loss: 1.2338e-04 - val_mse: 1.2338e-04
Epoch 4/25
186/186 [==============================] - 1s 4ms/step - loss: 6.2886e-05 - mse: 6.2886e-05 - val_loss: 7.3393e-05 - val_mse: 7.3393e-05
Epoch 5/25
186/186 [==============================] - 1s 4ms/step - loss: 6.1879e-05 - mse: 6.1879e-05 - val_loss: 6.4268e-05 - val_mse: 6.4268e-05
Epoch 6/25
186/186 [==============================] - 1s 4ms/step - loss: 5.4608e-05 - mse: 5.4608e-05 - val_loss: 6.1427e-05 - val_mse: 6.1427e-05
Epoch 7/25
186/186 [==============================] - 1s 4ms/step - loss: 5.2810e-05 - mse: 5.2810e-05 - val_los

array([[24319.334]], dtype=float32)

In [140]:
#Ethereum
coin = 'ethereum'
data = processData(coin)
scaler, X_train, y_train, X_test, y_test = preprocessData(data)
model_train(X_train, y_train, coin)
predict_NextDay(scaler, coin)

Epoch 1/25
186/186 [==============================] - 6s 9ms/step - loss: 8.9572e-04 - mse: 8.9572e-04 - val_loss: 1.1846e-04 - val_mse: 1.1846e-04
Epoch 2/25
186/186 [==============================] - 1s 4ms/step - loss: 1.3700e-04 - mse: 1.3700e-04 - val_loss: 1.8427e-04 - val_mse: 1.8427e-04
Epoch 3/25
186/186 [==============================] - 1s 4ms/step - loss: 1.0556e-04 - mse: 1.0556e-04 - val_loss: 7.0098e-05 - val_mse: 7.0098e-05
Epoch 4/25
186/186 [==============================] - 1s 4ms/step - loss: 9.8621e-05 - mse: 9.8621e-05 - val_loss: 4.6291e-05 - val_mse: 4.6291e-05
Epoch 5/25
186/186 [==============================] - 1s 4ms/step - loss: 1.2713e-04 - mse: 1.2713e-04 - val_loss: 1.0763e-04 - val_mse: 1.0763e-04
Epoch 6/25
186/186 [==============================] - 1s 4ms/step - loss: 7.9570e-05 - mse: 7.9570e-05 - val_loss: 8.4264e-05 - val_mse: 8.4264e-05
Epoch 7/25
186/186 [==============================] - 1s 5ms/step - loss: 8.1514e-05 - mse: 8.1514e-05 - val_los

array([[1936.8062]], dtype=float32)

In [141]:
#IOTA
coin = 'iota'
data = processData(coin)
scaler, X_train, y_train, X_test, y_test = preprocessData(data)
model_train(X_train, y_train, coin)
predict_NextDay(scaler, coin)

Epoch 1/25
168/168 [==============================] - 6s 10ms/step - loss: 0.0047 - mse: 0.0047 - val_loss: 3.6387e-04 - val_mse: 3.6387e-04
Epoch 2/25
168/168 [==============================] - 1s 4ms/step - loss: 7.5543e-04 - mse: 7.5543e-04 - val_loss: 1.3766e-04 - val_mse: 1.3766e-04
Epoch 3/25
168/168 [==============================] - 1s 4ms/step - loss: 5.7679e-04 - mse: 5.7679e-04 - val_loss: 1.0845e-04 - val_mse: 1.0845e-04
Epoch 4/25
168/168 [==============================] - 1s 4ms/step - loss: 5.9121e-04 - mse: 5.9121e-04 - val_loss: 3.8544e-04 - val_mse: 3.8544e-04
Epoch 5/25
168/168 [==============================] - 1s 4ms/step - loss: 5.8745e-04 - mse: 5.8745e-04 - val_loss: 9.1939e-05 - val_mse: 9.1939e-05
Epoch 6/25
168/168 [==============================] - 1s 4ms/step - loss: 4.9196e-04 - mse: 4.9196e-04 - val_loss: 9.0768e-05 - val_mse: 9.0768e-05
Epoch 7/25
168/168 [==============================] - 1s 4ms/step - loss: 4.3013e-04 - mse: 4.3013e-04 - val_loss: 8.77

array([[0.36208424]], dtype=float32)

In [142]:
#TRON
coin = 'tron'
data = processData(coin)
scaler, X_train, y_train, X_test, y_test = preprocessData(data)
model_train(X_train, y_train, coin)
predict_NextDay(scaler, coin)

Epoch 1/25
181/181 [==============================] - 7s 9ms/step - loss: 0.0042 - mse: 0.0042 - val_loss: 5.1660e-04 - val_mse: 5.1660e-04
Epoch 2/25
181/181 [==============================] - 1s 4ms/step - loss: 5.4623e-04 - mse: 5.4623e-04 - val_loss: 2.9341e-04 - val_mse: 2.9341e-04
Epoch 3/25
181/181 [==============================] - 1s 4ms/step - loss: 4.3270e-04 - mse: 4.3270e-04 - val_loss: 2.0217e-04 - val_mse: 2.0217e-04
Epoch 4/25
181/181 [==============================] - 1s 4ms/step - loss: 3.4397e-04 - mse: 3.4397e-04 - val_loss: 4.0900e-04 - val_mse: 4.0900e-04
Epoch 5/25
181/181 [==============================] - 1s 5ms/step - loss: 2.8291e-04 - mse: 2.8291e-04 - val_loss: 3.5901e-04 - val_mse: 3.5901e-04
Epoch 6/25
181/181 [==============================] - 1s 5ms/step - loss: 2.6597e-04 - mse: 2.6597e-04 - val_loss: 2.0836e-04 - val_mse: 2.0836e-04
Epoch 7/25
181/181 [==============================] - 1s 5ms/step - loss: 2.5166e-04 - mse: 2.5166e-04 - val_loss: 2.120

array([[0.06133265]], dtype=float32)